In [7]:
from naas_drivers import linkedin
import pandas as pd
import numpy as np
import naas

In [8]:
LI_AT = ''  # EXAMPLE AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = ''  # EXAMPLE ajax:8379907400220387585

In [9]:
#POST_URL = "https://www.linkedin.com/posts/calok64_api-activity-6872773401027588096-NTys"
#DF_post_stats = linkedin.connect(LI_AT, JSESSIONID).post.get_stats(POST_URL)
#DF_post_likes = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(POST_URL)

In [10]:
PROFILE_URL = "https://www.linkedin.com/in/calok64/"

In [11]:
DF_profile_posts = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_stats(PROFILE_URL)

In [12]:
DF_all_post_likes = pd.DataFrame()
for index, row in DF_profile_posts.iterrows():
    df = linkedin.connect(LI_AT, JSESSIONID).post.get_likes(row['POST_URL'])
    if not df.empty:
        DF_all_post_likes = DF_all_post_likes.append(df)

In [13]:
profile_posts = DF_all_post_likes[DF_all_post_likes['POST_URL'].str.contains('calok64')]

In [14]:
grouped_profile_posts = profile_posts.groupby(['PROFILE_ID']).size().sort_values(ascending=False).reset_index(name='count')

In [137]:
conditions = [
    (grouped_profile_posts['count'] <= 2),
    (grouped_profile_posts['count'] > 2) & (grouped_profile_posts['count'] <= 9),
    (grouped_profile_posts['count'] > 9) & (grouped_profile_posts['count'] <= 15),
    (grouped_profile_posts['count'] > 15)
    ]

values = ['L4', 'L3', 'L2', 'L1']

In [138]:
grouped_profile_posts['POTENTIAL_LEAD'] = np.select(conditions, values)

In [139]:
#PROFILE_ID
PROFILE_CONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(PROFILE_URL)
PROFILE_IDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(PROFILE_URL)

In [140]:
profile_Email = PROFILE_CONTACTS.at[0,'EMAIL']

In [141]:
profile_Name = PROFILE_IDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILE_IDENTITY.at[0,'LASTNAME']

In [142]:
leads_list = [];

r_count = 1
for index, row in grouped_profile_posts.iterrows():
    if row['POTENTIAL_LEAD'] == "L3" or row['POTENTIAL_LEAD'] == "L2" or row['POTENTIAL_LEAD'] == "L1":
        profileid = row['PROFILE_ID']
        profileurl = "https://www.linkedin.com/in/"+profileid+"/"
        PROFILECONTACTS = linkedin.connect(LI_AT, JSESSIONID).profile.get_contact(profileurl)
        PROFILEIDENTITY = linkedin.connect(LI_AT, JSESSIONID).profile.get_identity(profileurl)
        profileemail = PROFILECONTACTS.at[0,'EMAIL']
        profilename = PROFILEIDENTITY.at[0,'FIRSTNAME'] + " "+ PROFILEIDENTITY.at[0,'LASTNAME']
        if profileemail != None :
            leads_list.append([r_count,profilename,profileemail,profileurl])
            r_count = r_count + 1
   

In [143]:
leads_list_str = ""
for i in leads_list:
    leads_list_str = leads_list_str + "<tr><td style='width: 7.04591%;'>{}".format(i[0]) + "</td>" + "<td style='width: 25%;'>"+i[1]+"</td>" +"<td style='width: 25%;'>"+ i[2]+"</td>" +"<td style='width: 25%;'>"+ i[3]+"</td>" +"</tr>"  

In [145]:
email_to = "c.alok0493@gmail.com"
subject = "Linkedin Leads Alert"
content = f"<p>Hi there,</p><br/><p>Here is the list of the leads you generated from your posts on LinkedIN</p><br/><table style='border-collapse: collapse; width: 100%;' border='1'><tbody><tr><td style='width: 3%;'>Sr. No</td><td style='width: 25%;'>Full Name</td><td style='width: 25%;'>Email ID</td><td style='width: 25%;'>LinkedIN Profile</td></tr>" + leads_list_str + "</tbody></table>" + "<br/><p>You can see them in your Hubspot “Contact” section.</p><br/><br/><br/>Best Regards from Naas."

naas.notification.send(email_to=email_to, subject=subject, html=content)

👌 💌 Email has been sent successfully !
